In [1]:
def Soma(var1:float,var2:float)->int:
  """Esta função retorna o reseultado da soma entre dois números.
  var1: variável numérica
  var2: variável numérica
  resultado: resultado da soma de var1 e var2"""

  resultado = var1 + var2
  print('O resultado da soma é: ', resultado)

  if resultado > 10:
    print('Só Alegria!!')

    return resultado

In [2]:
Soma(1,10)

O resultado da soma é:  11
Só Alegria!!


11

Instalando Dependências

In [3]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

Configurando o Kernel Spark

In [4]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Explorandos os Dados

Lendo público de modelagem (tabela Principal)

In [6]:
app_train = spark.read.csv("/content/drive/MyDrive/SQLMMs/InputData/application_train.csv", inferSchema=True, header=True)

app_train.registerTempTable('app_train')

app_train.count()

307511

Lendo tabela transacional Bureau

In [7]:
bureau = spark.read.csv("/content/drive/MyDrive/SQLMMs/InputData/bureau.csv", inferSchema=True, header=True)

bureau.registerTempTable('bureau')

bureau.count()

1716428

Verificando variáveis da tabela princicpal

In [8]:
app_train.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

Listando variáveis tipo string

In [9]:
string_list = [item[0] for item in app_train.dtypes if item[1].startswith('string')]
string_list

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

Exercícios sobre a tabela de modelagem

Cálculo da volumetria e taxa de evento para cada domínio de cada uma das variáveis tipo string.

In [10]:
spark.sql("""
select
  NAME_CONTRACT_tYPE,
  round(100*avg(TARGET),2) as AVG_TARGET,
  count(*) as Volume

  from
    app_train
  group by
    NAME_CONTRACT_TYPE
  order by
    AVG_TARGET desc
    """).show()

+------------------+----------+------+
|NAME_CONTRACT_tYPE|AVG_TARGET|Volume|
+------------------+----------+------+
|        Cash loans|      8.35|278232|
|   Revolving loans|      5.48| 29279|
+------------------+----------+------+



Entendi que "AVG_TARGET" é a porcentagem de inadimplentes sobre o total de pessoas ("Volume") que realizaram empréstimos (em dinheiro ou rotativo).

Revolving loans, empéstimo rotativo, acho que refere-se ao uso de cartão de crédito. Este evento possui menor média de inadimplência e de empréstimos. Creio que seja pelo fato de que a modalidae de empréstimos em cartões de crédito cobram juros muito mais elevados do que outros tipos de empréstimos. No Brasil, os juros em cartões de crédito são próximos de 300 % ao ano.

In [11]:
spark.sql("""
            select 
              OCCUPATION_TYPE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              OCCUPATION_TYPE
            order by 
              AVG_TARGET desc
""").show(100)

+--------------------+----------+------+
|     OCCUPATION_TYPE|AVG_TARGET|Volume|
+--------------------+----------+------+
|  Low-skill Laborers|     17.15|  2093|
|             Drivers|     11.33| 18603|
|Waiters/barmen staff|     11.28|  1348|
|      Security staff|     10.74|  6721|
|            Laborers|     10.58| 55186|
|       Cooking staff|     10.44|  5946|
|         Sales staff|      9.63| 32102|
|      Cleaning staff|      9.61|  4653|
|       Realty agents|      7.86|   751|
|         Secretaries|      7.05|  1305|
|      Medicine staff|       6.7|  8537|
|Private service s...|       6.6|  2652|
|                null|      6.51| 96391|
|            IT staff|      6.46|   526|
|            HR staff|      6.39|   563|
|          Core staff|       6.3| 27570|
|            Managers|      6.21| 21371|
|High skill tech s...|      6.16| 11380|
|         Accountants|      4.83|  9813|
+--------------------+----------+------+



Neste caso trabalhadores com menor grau de habilidades técnicas apresentam elevado percentual de inadimplência.

In [12]:
spark.sql("""
            select 
              CODE_GENDER,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              CODE_GENDER
            order by 
              AVG_TARGET desc
""").show()

+-----------+----------+------+
|CODE_GENDER|AVG_TARGET|Volume|
+-----------+----------+------+
|          M|     10.14|105059|
|          F|       7.0|202448|
|        XNA|       0.0|     4|
+-----------+----------+------+



Os homens são mais inadimplentes do que as mulheres, ainda que elas tenham feito mais empréstimos.

In [13]:
spark.sql("""
select
  FLAG_OWN_CAR,
  round(100*avg(TARGET),2) as AVG_TARGET,
  count(*) as Volume

  from
    app_train
  group by
    FLAG_OWN_CAR
  order by
    AVG_TARGET desc
    """).show()

+------------+----------+------+
|FLAG_OWN_CAR|AVG_TARGET|Volume|
+------------+----------+------+
|           N|       8.5|202924|
|           Y|      7.24|104587|
+------------+----------+------+



A inadimplência é quase ao mesmo nível entre os que possuem e não carro, sendo o número de empréstimos significativamente maior para os que não tem carro.

In [14]:
spark.sql("""
            select 
              FLAG_OWN_REALTY,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              FLAG_OWN_REALTY
            order by 
              AVG_TARGET desc
""").show()

+---------------+----------+------+
|FLAG_OWN_REALTY|AVG_TARGET|Volume|
+---------------+----------+------+
|              N|      8.32| 94199|
|              Y|      7.96|213312|
+---------------+----------+------+



Proprietários de casa ou partamento realizaram empréstimos em mais do que o dobro do que os que não possuem imóveis, sendo que para ambos o nível de inadiplência é próximo de 8%.

In [15]:
spark.sql("""
            select 
              NAME_TYPE_SUITE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              NAME_TYPE_SUITE
            order by 
              AVG_TARGET desc
""").show()

+---------------+----------+------+
|NAME_TYPE_SUITE|AVG_TARGET|Volume|
+---------------+----------+------+
|        Other_B|      9.83|  1770|
|        Other_A|      8.78|   866|
|Group of people|      8.49|   271|
|  Unaccompanied|      8.18|248526|
|Spouse, partner|      7.87| 11370|
|         Family|      7.49| 40149|
|       Children|      7.38|  3267|
|           null|      5.42|  1292|
+---------------+----------+------+



Não percebo resolução significativa do nível de inadimplência estando o solicitante acompanhado ou não ou o tipo de companhia. O percentual de inadimplência varia entre 7 e 10 %, sendo que o maior número de empréstimos ocorreu para os não acompanhados.

In [16]:
spark.sql("""
            select 
              NAME_INCOME_TYPE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              NAME_INCOME_TYPE
            order by 
              AVG_TARGET desc
""").show()

+--------------------+----------+------+
|    NAME_INCOME_TYPE|AVG_TARGET|Volume|
+--------------------+----------+------+
|     Maternity leave|      40.0|     5|
|          Unemployed|     36.36|    22|
|             Working|      9.59|158774|
|Commercial associate|      7.48| 71617|
|       State servant|      5.75| 21703|
|           Pensioner|      5.39| 55362|
|             Student|       0.0|    18|
|         Businessman|       0.0|    10|
+--------------------+----------+------+



A maior inadimolência ocorreu para quem estava de licença maternidade e para desempregados, sendo estes ultimos os que mais realizaram empréstimos. Estudantes e businessman não foram inadimplentes.

In [17]:
spark.sql("""
            select 
              NAME_EDUCATION_TYPE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              NAME_EDUCATION_TYPE
            order by 
              AVG_TARGET desc
""").show()

+--------------------+----------+------+
| NAME_EDUCATION_TYPE|AVG_TARGET|Volume|
+--------------------+----------+------+
|     Lower secondary|     10.93|  3816|
|Secondary / secon...|      8.94|218391|
|   Incomplete higher|      8.48| 10277|
|    Higher education|      5.36| 74863|
|     Academic degree|      1.83|   164|
+--------------------+----------+------+



A inadimplência foi maior para aqueles com baixo grau de escolaridade, e de maneira oposta, menor para aqueles com curso superior.

In [18]:
spark.sql("""
            select 
              NAME_FAMILY_STATUS,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              NAME_FAMILY_STATUS
            order by 
              AVG_TARGET desc
""").show()

+--------------------+----------+------+
|  NAME_FAMILY_STATUS|AVG_TARGET|Volume|
+--------------------+----------+------+
|      Civil marriage|      9.94| 29775|
|Single / not married|      9.81| 45444|
|           Separated|      8.19| 19770|
|             Married|      7.56|196432|
|               Widow|      5.82| 16088|
|             Unknown|       0.0|     2|
+--------------------+----------+------+



O estado civil pouco afeta a inadimplência, sendo apeans um pouco menor para viúvos.

In [19]:
spark.sql("""
            select 
              NAME_HOUSING_TYPE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              NAME_HOUSING_TYPE
            order by 
              AVG_TARGET desc
""").show()

+-------------------+----------+------+
|  NAME_HOUSING_TYPE|AVG_TARGET|Volume|
+-------------------+----------+------+
|   Rented apartment|     12.31|  4881|
|       With parents|      11.7| 14840|
|Municipal apartment|      8.54| 11183|
|    Co-op apartment|      7.93|  1122|
|  House / apartment|       7.8|272868|
|   Office apartment|      6.57|  2617|
+-------------------+----------+------+



A inadimplência foi maior para aqueles com aparatamento alugado. 

In [20]:
spark.sql("""
            select 
              WEEKDAY_APPR_PROCESS_START,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              WEEKDAY_APPR_PROCESS_START
            order by 
              AVG_TARGET desc
""").show()

+--------------------------+----------+------+
|WEEKDAY_APPR_PROCESS_START|AVG_TARGET|Volume|
+--------------------------+----------+------+
|                   TUESDAY|      8.35| 53901|
|                 WEDNESDAY|      8.16| 51934|
|                    FRIDAY|      8.15| 50338|
|                  THURSDAY|       8.1| 50591|
|                    SUNDAY|      7.93| 16181|
|                  SATURDAY|      7.89| 33852|
|                    MONDAY|      7.76| 50714|
+--------------------------+----------+------+



O dia de aplicação do empréstimo não parece ser significativo quanto a inadimplência.

In [21]:
spark.sql("""
            select 
              ORGANIZATION_TYPE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              ORGANIZATION_TYPE
            order by 
              AVG_TARGET desc
""").show(200)

+--------------------+----------+------+
|   ORGANIZATION_TYPE|AVG_TARGET|Volume|
+--------------------+----------+------+
|   Transport: type 3|     15.75|  1187|
|   Industry: type 13|     13.43|    67|
|    Industry: type 8|      12.5|    24|
|          Restaurant|     11.71|  1811|
|        Construction|     11.68|  6721|
|            Cleaning|     11.15|   260|
|    Industry: type 1|     11.07|  1039|
|    Industry: type 3|     10.62|  3278|
|             Realtor|     10.61|   396|
|         Agriculture|     10.47|  2454|
|       Trade: type 3|     10.34|  3492|
|       Self-employed|     10.17| 38412|
|    Industry: type 4|     10.15|   877|
|            Security|      9.98|  3247|
|       Trade: type 7|      9.45|  7831|
|Business Entity T...|       9.3| 67992|
|   Transport: type 4|      9.28|  5398|
|              Mobile|      9.15|   317|
|       Trade: type 1|      8.91|   348|
|   Industry: type 11|      8.65|  2704|
|Business Entity T...|      8.53| 10553|
|              P

A inadimplência varia entre 8 e 16 % entre os diferentes tipos de empresas onde trabalham os clientes.

In [22]:
spark.sql("""
            select 
              FONDKAPREMONT_MODE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              FONDKAPREMONT_MODE
            order by 
              AVG_TARGET desc
""").show()

+--------------------+----------+------+
|  FONDKAPREMONT_MODE|AVG_TARGET|Volume|
+--------------------+----------+------+
|                null|      8.62|210295|
|       not specified|      7.54|  5687|
|    reg oper account|      6.98| 73830|
|reg oper spec acc...|      6.56| 12080|
|    org spec account|      5.82|  5619|
+--------------------+----------+------+



In [23]:
spark.sql("""
            select 
              HOUSETYPE_MODE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              HOUSETYPE_MODE
            order by 
              AVG_TARGET desc
""").show()

+----------------+----------+------+
|  HOUSETYPE_MODE|AVG_TARGET|Volume|
+----------------+----------+------+
|specific housing|     10.14|  1499|
|            null|      9.15|154297|
|  terraced house|       8.5|  1212|
|  block of flats|      6.94|150503|
+----------------+----------+------+



In [24]:
spark.sql("""
            select 
              WALLSMATERIAL_MODE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              WALLSMATERIAL_MODE
            order by 
              AVG_TARGET desc
""").show()

+------------------+----------+------+
|WALLSMATERIAL_MODE|AVG_TARGET|Volume|
+------------------+----------+------+
|            Wooden|       9.7|  5362|
|              null|      9.13|156341|
|            Others|      8.31|  1625|
|             Mixed|      7.53|  2296|
|      Stone, brick|      7.41| 64815|
|             Block|      7.02|  9253|
|             Panel|      6.35| 66040|
|        Monolithic|      4.72|  1779|
+------------------+----------+------+



In [25]:
spark.sql("""
            select 
              EMERGENCYSTATE_MODE,
              round(100*avg(TARGET),2) as AVG_TARGET,
              count(*) as Volume

            from 
              app_train
            group by 
              EMERGENCYSTATE_MODE
            order by 
              AVG_TARGET desc
""").show()

+-------------------+----------+------+
|EMERGENCYSTATE_MODE|AVG_TARGET|Volume|
+-------------------+----------+------+
|                Yes|      9.58|  2328|
|               null|      9.26|145755|
|                 No|      6.96|159428|
+-------------------+----------+------+



In [26]:
#tabela de modelagem
app_train.show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+------------

In [27]:
#tabela de bereau
bureau.show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    215354|     5714462|       Closed|     currency 1|       -497|                 0|             -153.0|           -153

In [28]:
#verificando a estrutursa da tabela bereau
bureau.printSchema()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- SK_ID_BUREAU: integer (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: integer (nullable = true)
 |-- CREDIT_DAY_OVERDUE: integer (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: double (nullable = true)
 |-- DAYS_ENDDATE_FACT: double (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: double (nullable = true)
 |-- CNT_CREDIT_PROLONG: integer (nullable = true)
 |-- AMT_CREDIT_SUM: double (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: double (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: double (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: double (nullable = true)
 |-- CREDIT_TYPE: string (nullable = true)
 |-- DAYS_CREDIT_UPDATE: integer (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)



Cálculo da volumetria de cada um dos domínios das variáveis do tipo string.

In [29]:
spark.sql("""
            select
              CREDIT_ACTIVE,
              count(*) as Volume

            from
              bureau
            group by
              CREDIT_ACTIVE
            order by
              Volume desc
""").show()

+-------------+-------+
|CREDIT_ACTIVE| Volume|
+-------------+-------+
|       Closed|1079273|
|       Active| 630607|
|         Sold|   6527|
|     Bad debt|     21|
+-------------+-------+



In [30]:
spark.sql("""
            Select
              count(*) N_linhas
            from bureau
""").show()

+--------+
|N_linhas|
+--------+
| 1716428|
+--------+



Comparando os dois últimos resultados é possível concluir que o domínio "Bad debit" não possui representatividade significativa sobre o negócio por consitutir uma volumetria de apenas 21 eventos frente a quase 2 milhões de linhas da tabela "bureau".

Tentando "trocar por miúdos": O problema de negócio deve ser explorado em função de indicadores preditivos, variáveis explicativas. O termo é bastante sugestivo e significa que deve-se identificar adequadamente ao problema de negócio em questão, as variáveis numéricas que representam o fenômeno. Neste caso, a probabilidade de quitação de um empréstimo. 

A recomendação é que seja identificado o parâmetro dentre os dados que exercem grande influência sobre esses.

In [31]:
# carregar dados da tabela metadados
metadados = spark.read.csv("/content/drive/MyDrive/SQLMMs/InputData/HomeCredit_columns_description.csv", inferSchema=True, header=True)

metadados.registerTempTable('metadados')

metadados.count()

219

In [32]:
spark.sql("""
            select
              *
            from
              metadados
            where Table = 'bureau.csv'
""").show(30,False)

+---+----------+----------------------+-------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|_c0|Table     |Row                   |Description                                                                                                        |Special                              |
+---+----------+----------------------+-------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|125|bureau.csv|SK_ID_CURR            |ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau |hashed                               |
|126|bureau.csv|SK_BUREAU_ID          |Recoded ID of previous Credit Bureau credit related to our loan (unique coding for each loan application)          |hashed                               |
|127|bureau.csv|CREDIT_ACTIVE 

Criando um nova tabela para a nova variável.

In [33]:
df_bureau_01 = spark.sql("""
                          select
                            SK_ID_CURR,
                            round(avg(DAYS_CREDIT),2) as AVG_DAYS_CREIT

                          from
                            bureau
                          group by
                            SK_ID_CURR
""")

df_bureau_01.registerTempTable('df_bureau_01')

df_bureau_01.count()

305811

Naturalmente a operação de média sobre a variável "DAYS_CREDIT" produz uma única saída correspondente a cada indivíduo SK_ID_CURR. Esse processo de agregação entre a tabela principal e a tabela de transações (Bureau) iguala a granulometria dessas tabelas.

In [34]:
spark.sql("""
            select
              *
            from
              df_bureau_01
            where
              SK_ID_CURR = 100002
""").show()

+----------+--------------+
|SK_ID_CURR|AVG_DAYS_CREIT|
+----------+--------------+
|    100002|        -874.0|
+----------+--------------+



Por que o resultado é negativo (-874.0)?
Significa que o novo crédito foi solicitado com média de 874 dias antes da quitação do empréstimo corrente (vigente, em curso).

In [35]:
#Exibindo a tabela df_bureau_01
df_bureau_01.show()

+----------+--------------+
|SK_ID_CURR|AVG_DAYS_CREIT|
+----------+--------------+
|    341504|        -283.5|
|    197603|      -1916.43|
|    330299|       -1645.0|
|    355377|       -1540.0|
|    454739|      -1215.86|
|    295286|      -1264.89|
|    192082|      -1451.21|
|    109608|       -504.33|
|    375705|       -896.75|
|    380065|       -638.67|
|    416191|       -1429.0|
|    162260|       -1507.4|
|    166160|        -856.0|
|    191350|        -151.0|
|    258865|        -979.5|
|    439807|        -795.2|
|    258129|       -434.75|
|    133018|      -1261.64|
|    338173|      -1318.24|
|    338091|       -1022.0|
+----------+--------------+
only showing top 20 rows



In [36]:
#Exibindo os valores mínimos e máximos da variável "DAYS_CREDIT" para cada valor de "SK_ID_CURR"
spark.sql("""
            Select
              SK_ID_CURR,
              min(DAYS_CREDIT) as MIN_DAYS_CREDIT,
              max(DAYS_CREDIT) as MAX_DAYS_CREDIT
            from
              bureau
            group by
              SK_ID_CURR
            order by
              SK_ID_CURR
""").show()


+----------+---------------+---------------+
|SK_ID_CURR|MIN_DAYS_CREDIT|MAX_DAYS_CREDIT|
+----------+---------------+---------------+
|    100001|          -1572|            -49|
|    100002|          -1437|           -103|
|    100003|          -2586|           -606|
|    100004|          -1326|           -408|
|    100005|           -373|            -62|
|    100007|          -1149|          -1149|
|    100008|          -1097|            -78|
|    100009|          -2882|           -239|
|    100010|          -2741|          -1138|
|    100011|          -2508|          -1309|
|    100013|          -2070|          -1210|
|    100014|          -2308|           -376|
|    100015|          -1409|           -319|
|    100016|          -1634|           -128|
|    100017|          -2881|           -909|
|    100019|           -495|           -495|
|    100020|           -492|            -90|
|    100022|           -385|           -289|
|    100023|          -2624|           -157|
|    10002

In [37]:
# Inserindo os valores de mínimo e máximo anteriores na tabela "df_bureau_a1"
df_bureau_01 = spark.sql("""
                          select
                            SK_ID_CURR,
                            min(DAYS_CREDIT) as MIN_DAYS_CREDIT,
                            max(DAYS_CREDIT) as MAX_DAYS_CREDIT,
                            round(avg(DAYS_CREDIT),2) as AVG_DAYS_CREIT

                          from
                            bureau
                          group by
                            SK_ID_CURR
""")

df_bureau_01.registerTempTable('df_bureau_01')

df_bureau_01.count()

305811

In [38]:
# Mostrando a tabela gerada anteriormente.
df_bureau_01.show()

+----------+---------------+---------------+--------------+
|SK_ID_CURR|MIN_DAYS_CREDIT|MAX_DAYS_CREDIT|AVG_DAYS_CREIT|
+----------+---------------+---------------+--------------+
|    341504|           -285|           -282|        -283.5|
|    197603|          -2836|           -594|      -1916.43|
|    330299|          -2619|            -98|       -1645.0|
|    355377|          -2800|           -859|       -1540.0|
|    454739|          -2718|           -157|      -1215.86|
|    295286|          -2617|           -212|      -1264.89|
|    192082|          -2818|           -199|      -1451.21|
|    109608|          -1293|            -40|       -504.33|
|    375705|          -1179|           -623|       -896.75|
|    380065|          -1098|           -409|       -638.67|
|    416191|          -2891|           -640|       -1429.0|
|    162260|          -2849|           -185|       -1507.4|
|    166160|           -890|           -822|        -856.0|
|    191350|           -151|           -

In [39]:
#Verificando os valores de mínimo, máximo e média das variáveis numéricas contidas na tabela Bureau
spark.sql("""
            Select
              SK_ID_CURR,
              min(CREDIT_DAY_OVERDUE) as MIN_CREDIT_DAY_OVERDUE,
              max(CREDIT_DAY_OVERDUE) as MAX_CREDIT_DAY_OVERDUE,
              round(avg(CREDIT_DAY_OVERDUE),2) as AVG_CREDIT_DAY_OVERDUE,
              min(DAYS_CREDIT_ENDDATE) as MIN_DAYS_CREDIT_ENDDATE,
              max(DAYS_CREDIT_ENDDATE) as MAX_DAYS_CREDIT_ENDDATE,
              round(avg(DAYS_CREDIT_ENDDATE),2) as AVG_DAYS_CREDIT_ENDDATE,
              min(DAYS_ENDDATE_FACT) as MIN_DAYS_ENDDATE_FACT,
              max(DAYS_ENDDATE_FACT) as MAX_DAYS_ENDDATE_FACT,
              round(avg(DAYS_ENDDATE_FACT),2) as AVG_DAYS_ENDDATE_FACT,
              min(AMT_CREDIT_MAX_OVERDUE) as MIN_AMT_CREDIT_MAX_OVERDUE,
              max(AMT_CREDIT_MAX_OVERDUE) as MAX_AMT_CREDIT_MAX_OVERDUE,
              round(avg(AMT_CREDIT_MAX_OVERDUE),2) as AVG_AMT_CREDIT_MAX_OVERDUE,
              min(CNT_CREDIT_PROLONG) as MIN_CNT_CREDIT_PROLONG,
              max(CNT_CREDIT_PROLONG) as MAX_CNT_CREDIT_PROLONG,
              round(avg(CNT_CREDIT_PROLONG),2) as AVG_CNT_CREDIT_PROLONG,
              min(AMT_CREDIT_SUM) as MIN_AMT_CREDIT_SUM,
              max(AMT_CREDIT_SUM) as MAX_AMT_CREDIT_SUM,
              round(avg(AMT_CREDIT_SUM),2) as AVG_AMT_CREDIT_SUM,
              min(AMT_CREDIT_SUM_DEBT) as MIN_AMT_CREDIT_SUM_DEBT,
              max(AMT_CREDIT_SUM_DEBT) as MAX_AMT_CREDIT_SUM_DEBT,
              round(avg(AMT_CREDIT_SUM_DEBT),2) as AVG_AMT_CREDIT_SUM_DEBT,
              min(AMT_CREDIT_SUM_LIMIT) as MIN_AMT_CREDIT_SUM_LIMIT,
              max(AMT_CREDIT_SUM_LIMIT) as MAX_AMT_CREDIT_SUM_LIMIT,
              round(avg(AMT_CREDIT_SUM_LIMIT),2) as AVG_AMT_CREDIT_SUM_LIMIT,
              min(AMT_CREDIT_SUM_OVERDUE) as MIN_AMT_CREDIT_SUM_OVERDUE,
              max(AMT_CREDIT_SUM_OVERDUE) as MAX_AMT_CREDIT_SUM_OVERDUE,
              round(avg(AMT_CREDIT_SUM_OVERDUE),2) as AVG_AMT_CREDIT_SUM_OVERDUE,
              min(DAYS_CREDIT_UPDATE) as MIN_DAYS_CREDIT_UPDATE,
              max(DAYS_CREDIT_UPDATE) as MAX_DAYS_CREDIT_UPDATE,
              round(avg(DAYS_CREDIT_UPDATE),2) as AVG_DAYS_CREDIT_UPDATE,
              min(AMT_ANNUITY) as MIN_DAYS_AMT_ANNUITY,
              max(AMT_ANNUITY) as MAX_DAYS_AMT_ANNUITY,
              round(avg(AMT_ANNUITY),2) as AVG_AMT_ANNUITY

            from
              bureau
            group by
              SK_ID_CURR
            order by
              SK_ID_CURR
""").show()

+----------+----------------------+----------------------+----------------------+-----------------------+-----------------------+-----------------------+---------------------+---------------------+---------------------+--------------------------+--------------------------+--------------------------+----------------------+----------------------+----------------------+------------------+------------------+------------------+-----------------------+-----------------------+-----------------------+------------------------+------------------------+------------------------+--------------------------+--------------------------+--------------------------+----------------------+----------------------+----------------------+--------------------+--------------------+---------------+
|SK_ID_CURR|MIN_CREDIT_DAY_OVERDUE|MAX_CREDIT_DAY_OVERDUE|AVG_CREDIT_DAY_OVERDUE|MIN_DAYS_CREDIT_ENDDATE|MAX_DAYS_CREDIT_ENDDATE|AVG_DAYS_CREDIT_ENDDATE|MIN_DAYS_ENDDATE_FACT|MAX_DAYS_ENDDATE_FACT|AVG_DAYS_ENDDATE_FACT

In [40]:
# Inserindo os valores de mínimo, máximo e média das variáveis numéricas da tabela "bureau" na tabela "df_bureau_01"
df_bureau_01 = spark.sql("""
                          select
                            SK_ID_CURR,
                            min(DAYS_CREDIT) as MIN_DAYS_CREDIT,
                            max(DAYS_CREDIT) as MAX_DAYS_CREDIT,
                            round(avg(DAYS_CREDIT),2) as AVG_DAYS_CREIT,
                            min(CREDIT_DAY_OVERDUE) as MIN_CREDIT_DAY_OVERDUE,
                            max(CREDIT_DAY_OVERDUE) as MAX_CREDIT_DAY_OVERDUE,
                            round(avg(CREDIT_DAY_OVERDUE),2) as AVG_CREDIT_DAY_OVERDUE,
                            min(DAYS_CREDIT_ENDDATE) as MIN_DAYS_CREDIT_ENDDATE,
                            max(DAYS_CREDIT_ENDDATE) as MAX_DAYS_CREDIT_ENDDATE,
                            round(avg(DAYS_CREDIT_ENDDATE),2) as AVG_DAYS_CREDIT_ENDDATE,
                            min(DAYS_ENDDATE_FACT) as MIN_DAYS_ENDDATE_FACT,
                            max(DAYS_ENDDATE_FACT) as MAX_DAYS_ENDDATE_FACT,
                            round(avg(DAYS_ENDDATE_FACT),2) as AVG_DAYS_ENDDATE_FACT,
                            min(AMT_CREDIT_MAX_OVERDUE) as MIN_AMT_CREDIT_MAX_OVERDUE,
                            max(AMT_CREDIT_MAX_OVERDUE) as MAX_AMT_CREDIT_MAX_OVERDUE,
                            round(avg(AMT_CREDIT_MAX_OVERDUE),2) as AVG_AMT_CREDIT_MAX_OVERDUE,
                            min(CNT_CREDIT_PROLONG) as MIN_CNT_CREDIT_PROLONG,
                            max(CNT_CREDIT_PROLONG) as MAX_CNT_CREDIT_PROLONG,
                            round(avg(CNT_CREDIT_PROLONG),2) as AVG_CNT_CREDIT_PROLONG,
                            min(AMT_CREDIT_SUM) as MIN_AMT_CREDIT_SUM,
                            max(AMT_CREDIT_SUM) as MAX_AMT_CREDIT_SUM,
                            round(avg(AMT_CREDIT_SUM),2) as AVG_AMT_CREDIT_SUM,
                            min(AMT_CREDIT_SUM_DEBT) as MIN_AMT_CREDIT_SUM_DEBT,
                            max(AMT_CREDIT_SUM_DEBT) as MAX_AMT_CREDIT_SUM_DEBT,
                            round(avg(AMT_CREDIT_SUM_DEBT),2) as AVG_AMT_CREDIT_SUM_DEBT,
                            min(AMT_CREDIT_SUM_LIMIT) as MIN_AMT_CREDIT_SUM_LIMIT,
                            max(AMT_CREDIT_SUM_LIMIT) as MAX_AMT_CREDIT_SUM_LIMIT,
                            round(avg(AMT_CREDIT_SUM_LIMIT),2) as AVG_AMT_CREDIT_SUM_LIMIT,
                            min(AMT_CREDIT_SUM_OVERDUE) as MIN_AMT_CREDIT_SUM_OVERDUE,
                            max(AMT_CREDIT_SUM_OVERDUE) as MAX_AMT_CREDIT_SUM_OVERDUE,
                            round(avg(AMT_CREDIT_SUM_OVERDUE),2) as AVG_AMT_CREDIT_SUM_OVERDUE,
                            min(DAYS_CREDIT_UPDATE) as MIN_DAYS_CREDIT_UPDATE,
                            max(DAYS_CREDIT_UPDATE) as MAX_DAYS_CREDIT_UPDATE,
                            round(avg(DAYS_CREDIT_UPDATE),2) as AVG_DAYS_CREDIT_UPDATE,
                            min(AMT_ANNUITY) as MIN_DAYS_AMT_ANNUITY,
                            max(AMT_ANNUITY) as MAX_DAYS_AMT_ANNUITY,
                            round(avg(AMT_ANNUITY),2) as AVG_AMT_ANNUITY

                          from
                            bureau
                          group by
                            SK_ID_CURR
""")

df_bureau_01.registerTempTable('df_bureau_01')

df_bureau_01.count()

305811

In [41]:
# Exibindo toda as colunas da tabela "df_bureau_01".
df_bureau_01.show(100,False)

+----------+---------------+---------------+--------------+----------------------+----------------------+----------------------+-----------------------+-----------------------+-----------------------+---------------------+---------------------+---------------------+--------------------------+--------------------------+--------------------------+----------------------+----------------------+----------------------+------------------+------------------+------------------+-----------------------+-----------------------+-----------------------+------------------------+------------------------+------------------------+--------------------------+--------------------------+--------------------------+----------------------+----------------------+----------------------+--------------------+--------------------+---------------+
|SK_ID_CURR|MIN_DAYS_CREDIT|MAX_DAYS_CREDIT|AVG_DAYS_CREIT|MIN_CREDIT_DAY_OVERDUE|MAX_CREDIT_DAY_OVERDUE|AVG_CREDIT_DAY_OVERDUE|MIN_DAYS_CREDIT_ENDDATE|MAX_DAYS_CREDIT_END

In [42]:
#Filtrando o indivíduo 100002
spark.sql("""
            Select
              *
            from
              df_bureau_01
            where
              SK_ID_CURR = 100002
""").show()

+----------+---------------+---------------+--------------+----------------------+----------------------+----------------------+-----------------------+-----------------------+-----------------------+---------------------+---------------------+---------------------+--------------------------+--------------------------+--------------------------+----------------------+----------------------+----------------------+------------------+------------------+------------------+-----------------------+-----------------------+-----------------------+------------------------+------------------------+------------------------+--------------------------+--------------------------+--------------------------+----------------------+----------------------+----------------------+--------------------+--------------------+---------------+
|SK_ID_CURR|MIN_DAYS_CREDIT|MAX_DAYS_CREDIT|AVG_DAYS_CREIT|MIN_CREDIT_DAY_OVERDUE|MAX_CREDIT_DAY_OVERDUE|AVG_CREDIT_DAY_OVERDUE|MIN_DAYS_CREDIT_ENDDATE|MAX_DAYS_CREDIT_END

Indicadores preditivos considerando as regras de negócio.

In [92]:
#Estes indicadores aqui, serão criados em outra tabela, "df_bureau_02"

df_bureau_02 = spark.sql("""
                          Select
                            SK_ID_CURR,
                            round(avg(abs(DAYS_CREDIT)),2) as AVG_DAYS_CREDIT,
                            round(avg(case when CREDIT_ACTIVE in ('Active') then abs(DAYS_CREDIT) else 0 end),2) as AVG_CREDIT_ACTV_ACTIVE,
                            round(avg(case when CREDIT_ACTIVE in ('Closed') then abs(DAYS_CREDIT) else 0 end),2) as AVG_CREDIT_ACTV_CLOSED,
                            round(avg(case when CREDIT_ACTIVE in ('Sold') then abs(DAYS_CREDIT) else 0 end),2) as AVG_CREDIT_ACTV_SOLD,
                            round(avg(case when CREDIT_ACTIVE in ('Bad debit') then abs(DAYS_CREDIT) else 0 end),2) as AVG_CREDIT_ACTV_BADDBT,

                            round(sum(abs(DAYS_CREDIT)),2) as SUM_DAYS_CREDIT,
                            round(sum(case when CREDIT_ACTIVE in ('Active') then abs(DAYS_CREDIT) else 0 end),2) as SUM_CREDIT_ACTV_ACTIVE,
                            round(sum(case when CREDIT_ACTIVE in ('Closed') then abs(DAYS_CREDIT) else 0 end),2) as SUM_CREDIT_ACTV_CLOSED,
                            round(sum(case when CREDIT_ACTIVE in ('Sold') then abs(DAYS_CREDIT) else 0 end),2) as SUM_CREDIT_ACTV_SOLD,
                            round(sum(case when CREDIT_ACTIVE in ('Bad debit') then abs(DAYS_CREDIT) else 0 end),2) as SUM_CREDIT_ACTV_BADDBT,

                            round(min(abs(DAYS_CREDIT)),2) as MIN_DAYS_CREDIT,
                            round(min(case when CREDIT_ACTIVE in ('Active') then abs(DAYS_CREDIT) else 0 end),2) as MIN_CREDIT_ACTV_ACTIVE,
                            round(min(case when CREDIT_ACTIVE in ('Closed') then abs(DAYS_CREDIT) else 0 end),2) as MIN_CREDIT_ACTV_CLOSED,
                            round(min(case when CREDIT_ACTIVE in ('Sold') then abs(DAYS_CREDIT) else 0 end),2) as MIN_CREDIT_ACTV_SOLD,
                            round(min(case when CREDIT_ACTIVE in ('Bad debit') then abs(DAYS_CREDIT) else 0 end),2) as MIN_CREDIT_ACTV_BADDBT,

                            
                            round(avg(((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100)),2) as AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM,
                            round(avg(case when CREDIT_ACTIVE in ('Active') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_ACTIVE,
                            round(avg(case when CREDIT_ACTIVE in ('Closed') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_CLOSED,
                            round(avg(case when CREDIT_ACTIVE in ('Sold') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_SOLD,
                            round(avg(case when CREDIT_ACTIVE in ('Bad debit') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_BADDBT,

                            round(min(((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100)),2) as MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM,
                            round(min(case when CREDIT_ACTIVE in ('Active') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_ACTIVE,
                            round(min(case when CREDIT_ACTIVE in ('Closed') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_CLOSED,
                            round(min(case when CREDIT_ACTIVE in ('Sold') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_SOLD,
                            round(min(case when CREDIT_ACTIVE in ('Bad debit') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_BADDBT,

                            round(max(((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100)),2) as MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM,
                            round(max(case when CREDIT_ACTIVE in ('Active') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_ACTIVE,
                            round(max(case when CREDIT_ACTIVE in ('Closed') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_CLOSED,
                            round(max(case when CREDIT_ACTIVE in ('Sold') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_SOLD,
                            round(max(case when CREDIT_ACTIVE in ('Bad debit') then ((AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM) * 100) else 0 end),1) as MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_BADDBT,

                            round(avg(((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100)),2) as AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM,
                            round(avg(case when CREDIT_ACTIVE in ('Active') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_ACTIVE,
                            round(avg(case when CREDIT_ACTIVE in ('Closed') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_CLOSED,
                            round(avg(case when CREDIT_ACTIVE in ('Sold') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_SOLD,
                            round(avg(case when CREDIT_ACTIVE in ('Bad debit') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_BADDBT,

                            round(min(((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100)),2) as MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM,
                            round(min(case when CREDIT_ACTIVE in ('Active') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_ACTIVE,
                            round(min(case when CREDIT_ACTIVE in ('Closed') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_CLOSED,
                            round(min(case when CREDIT_ACTIVE in ('Sold') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_SOLD,
                            round(min(case when CREDIT_ACTIVE in ('Bad debit') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_BADDBT,

                            round(max(((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100)),2) as MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM,
                            round(max(case when CREDIT_ACTIVE in ('Active') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_ACTIVE,
                            round(max(case when CREDIT_ACTIVE in ('Closed') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_CLOSED,
                            round(max(case when CREDIT_ACTIVE in ('Sold') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_SOLD,
                            round(max(case when CREDIT_ACTIVE in ('Bad debit') then ((AMT_CREDIT_SUM_DEBT - AMT_CREDIT_SUM) * 100) else 0 end),1) as MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_BADDBT

                          from
                            bureau
                          group by
                            SK_ID_CURR
""")

df_bureau_02.registerTempTable('df_bureau_02')

df_bureau_02.count()

305811

Verificando a volumetria e os dados gerados na tabela "df_bureau_02"

In [93]:
df_bureau_02.show()
df_bureau_02.count()

+----------+---------------+----------------------+----------------------+--------------------+----------------------+---------------+----------------------+----------------------+--------------------+----------------------+---------------+----------------------+----------------------+--------------------+----------------------+-----------------------------------------+------------------------------------------------+------------------------------------------------+----------------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+------------------------------------------------+----------------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+------------------------------------------------+----------------------------------------------+-----------

305811

In [94]:
#Filtrando o evento 100002
spark.sql("""
            select
              *
            from
              df_bureau_02
            where
              SK_ID_CURR = 100002
""").show()

+----------+---------------+----------------------+----------------------+--------------------+----------------------+---------------+----------------------+----------------------+--------------------+----------------------+---------------+----------------------+----------------------+--------------------+----------------------+-----------------------------------------+------------------------------------------------+------------------------------------------------+----------------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+------------------------------------------------+----------------------------------------------+------------------------------------------------+-----------------------------------------+------------------------------------------------+------------------------------------------------+----------------------------------------------+-----------

In [95]:
#Inscript para auxiliar na geração de códigos SQL
features_expl_list = df_bureau_02.columns
for feat in features_expl_list:
  var = 'b.' + feat + ','
  print(var)

b.SK_ID_CURR,
b.AVG_DAYS_CREDIT,
b.AVG_CREDIT_ACTV_ACTIVE,
b.AVG_CREDIT_ACTV_CLOSED,
b.AVG_CREDIT_ACTV_SOLD,
b.AVG_CREDIT_ACTV_BADDBT,
b.SUM_DAYS_CREDIT,
b.SUM_CREDIT_ACTV_ACTIVE,
b.SUM_CREDIT_ACTV_CLOSED,
b.SUM_CREDIT_ACTV_SOLD,
b.SUM_CREDIT_ACTV_BADDBT,
b.MIN_DAYS_CREDIT,
b.MIN_CREDIT_ACTV_ACTIVE,
b.MIN_CREDIT_ACTV_CLOSED,
b.MIN_CREDIT_ACTV_SOLD,
b.MIN_CREDIT_ACTV_BADDBT,
b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM,
b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_ACTIVE,
b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_CLOSED,
b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_SOLD,
b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_BADDBT,
b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM,
b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_ACTIVE,
b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_CLOSED,
b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_SOLD,
b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_BADDBT,
b.MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM,
b.MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_ACTIVE,
b.MAX_AMT_CREDIT_SUM_DEB

In [104]:
#LEFT JOIN entre as trabelas princial e variáveis preditivas do bureau
abt_bureau = spark.sql("""
                        Select

                          a.*,
                          b.AVG_DAYS_CREDIT,
                          b.AVG_CREDIT_ACTV_ACTIVE,
                          b.AVG_CREDIT_ACTV_CLOSED,
                          b.AVG_CREDIT_ACTV_SOLD,
                          b.AVG_CREDIT_ACTV_BADDBT,
                          b.SUM_DAYS_CREDIT,
                          b.SUM_CREDIT_ACTV_ACTIVE,
                          b.SUM_CREDIT_ACTV_CLOSED,
                          b.SUM_CREDIT_ACTV_SOLD,
                          b.SUM_CREDIT_ACTV_BADDBT,
                          b.MIN_DAYS_CREDIT,
                          b.MIN_CREDIT_ACTV_ACTIVE,
                          b.MIN_CREDIT_ACTV_CLOSED,
                          b.MIN_CREDIT_ACTV_SOLD,
                          b.MIN_CREDIT_ACTV_BADDBT,
                          b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM,
                          b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_ACTIVE,
                          b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_CLOSED,
                          b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_SOLD,
                          b.AVG_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_BADDBT,
                          b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM,
                          b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_ACTIVE,
                          b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_CLOSED,
                          b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_SOLD,
                          b.MIN_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_BADDBT,
                          b.MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM,
                          b.MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_ACTIVE,
                          b.MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_CLOSED,
                          b.MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_SOLD,
                          b.MAX_AMT_CREDIT_SUM_DEBT_by_AMT_CREDIT_SUM_BADDBT,
                          b.AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM,
                          b.AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_ACTIVE,
                          b.AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_CLOSED,
                          b.AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_SOLD,
                          b.AVG_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_BADDBT,
                          b.MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM,
                          b.MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_ACTIVE,
                          b.MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_CLOSED,
                          b.MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_SOLD,
                          b.MIN_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_BADDBT,
                          b.MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM,
                          b.MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_ACTIVE,
                          b.MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_CLOSED,
                          b.MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_SOLD,
                          b.MAX_AMT_CREDIT_SUM_DEBT_minus_AMT_CREDIT_SUM_BADDBT

                        from
                          app_train as a
                        left join
                          df_bureau_02 as b
                        on
                          a.SK_ID_CURR = b.SK_ID_CURR

""")

abt_bureau.registerTempTable('abt_bureau')

abt_bureau.show()

abt_bureau.count()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+--------------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+-------------------+-------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------

307511

In [105]:
# Salvando o tabela gerada no diretório das ABTs

abt_bureau.write.mode('overwrite').parquet("/content/drive/MyDrive/SQLMMs/ABT/VARS_BUREAU")


In [106]:
#Leitura dos arquivos salvos, verificação.

read_abt_bureau = spark.read.format("parquet").load("/content/drive/MyDrive/SQLMMs/ABT/VARS_BUREAU")

read_abt_bureau.show()

read_abt_bureau.count()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-------------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+--------------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+------------------+-------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+---------

307511